In [1]:
import pandas as pd
import os
import json
import requests

In [2]:
pathDataset = "C:/Users/jhpark/Desktop/개발/CODE/public-late-night-pharmacies/dataset"
os.chdir(pathDataset)

In [3]:
apiKey = "ef46baec5e4c75cbcbabc3b6b241ff4e" # 카카오 API키

# kakao 주소->좌표 변환 api 호출 함수
def addrressToCoord(addr):
    url = f'https://dapi.kakao.com/v2/local/search/address.json?query={addr}'
    headers = {"Authorization": "KakaoAK " + apiKey}
    result = json.loads(str(requests.get(url, headers=headers).text))
    if len(result['documents']) != 0:
        matchFirst = result['documents'][0]['address']
        return float(matchFirst['x']), float(matchFirst['y'])
    else:
        return None

In [4]:
# 데이터프레임 내 도로명주소 좌표 변환 함수
def dfAddrToCoordMap(df, columnName):
    sucess = 0
    error = 0
    for i in range(len(df)):
        addr = df.loc[i, columnName]
        print(f"주소: {addr}", end="")
        coord = addrressToCoord(addr)
        if coord is None:
            error += 1
            continue
        print(f" => 경도: {coord[0]}, 위도: {coord[1]}")
        df.loc[i, "X"] = coord[0]
        df.loc[i, "Y"] = coord[1]
        sucess += 1

    print(f"총 데이터 개수: {sucess + error}, 성공: {sucess}, 실패: {error}")

## 숙박업소 데이터

In [5]:
df_hotel = pd.read_csv("제주특별자치도_숙박업현황_20200102.csv", encoding='cp949')
df_hotel.info()
df_hotel.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1335 entries, 0 to 1334
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   행정시      1335 non-null   object
 1   업소명      1335 non-null   object
 2   소재지      1335 non-null   object
 3   전화번호     1201 non-null   object
 4   데이터기준일자  1335 non-null   object
dtypes: object(5)
memory usage: 52.3+ KB


,행정시,업소명,소재지,전화번호,데이터기준일자
0,서귀포시,위(WE)호텔,제주특별자치도 서귀포시 1100로 453-95,064-730-1200,2020-01-02
1,서귀포시,이에스제주리조트,제주특별자치도 서귀포시 1100로 501,070-4548-0285,2020-01-02
2,서귀포시,천백 휴양펜션,제주특별자치도 서귀포시 1100로 6,064-738-3234,2020-01-02
3,서귀포시,모이라펜션,제주특별자치도 서귀포시 516로 296,064-732-0005,2020-01-02
4,서귀포시,빅토르힐 리조트,제주특별자치도 서귀포시 516로277번길 45,064-780-6300,2020-01-02


In [6]:
# 중복값 제거
df_hotel.drop_duplicates(ignore_index=True, inplace=True)
df_hotel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1335 entries, 0 to 1334
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   행정시      1335 non-null   object
 1   업소명      1335 non-null   object
 2   소재지      1335 non-null   object
 3   전화번호     1201 non-null   object
 4   데이터기준일자  1335 non-null   object
dtypes: object(5)
memory usage: 52.3+ KB


In [7]:
# 좌표 변환
df_hotel = df_hotel[['행정시','업소명', '소재지']]
dfAddrToCoordMap(df_hotel, "소재지")

주소: 제주특별자치도 서귀포시 1100로 453-95 => 경도: 126.444618497938, 위도: 33.2858507894235
주소: 제주특별자치도 서귀포시 1100로 501 => 경도: 126.456559491066, 위도: 33.289778187496
주소: 제주특별자치도 서귀포시 1100로 6 => 경도: 126.431903584078, 위도: 33.2514487168738
주소: 제주특별자치도 서귀포시 516로 296 => 경도: 126.587218749742, 위도: 33.2790671766401
주소: 제주특별자치도 서귀포시 516로277번길 45 => 경도: 126.581436643284, 위도: 33.2793474532374
주소: 제주특별자치도 서귀포시 검은여로130번길 73 => 경도: 126.589634606266, 위도: 33.2492577484853
주소: 제주특별자치도 서귀포시 구산봉로21번길 62 => 경도: 126.453531155668, 위도: 33.2539280836047
주소: 제주특별자치도 서귀포시 김정문화로 15 => 경도: 126.506658454463, 위도: 33.2523851686521
주소: 제주특별자치도 서귀포시 김정문화로 51 => 경도: 126.510294984645, 위도: 33.2514240710323
주소: 제주특별자치도 서귀포시 김정문화로27번길 14-4 => 경도: 126.508345224108, 위도: 33.2528717847008
주소: 제주특별자치도 서귀포시 김정문화로27번길 23-3 => 경도: 126.507950602442, 위도: 33.2539317994981
주소: 제주특별자치도 서귀포시 김정문화로27번길 9-1 => 경도: 126.507548268061, 위도: 33.2524815629392
주소: 제주특별자치도 서귀포시 김정문화로41번길 10-3 => 경도: 126.509781134443, 위도: 33.2523369277521
주소: 제주특별자치도 서귀포시 김정문화로41번길 

In [8]:
df_hotel.info()
df_hotel.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1335 entries, 0 to 1334
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   행정시     1335 non-null   object 
 1   업소명     1335 non-null   object 
 2   소재지     1335 non-null   object 
 3   X       1330 non-null   float64
 4   Y       1330 non-null   float64
dtypes: float64(2), object(3)
memory usage: 52.3+ KB


,행정시,업소명,소재지,X,Y
0,서귀포시,위(WE)호텔,제주특별자치도 서귀포시 1100로 453-95,126.444618,33.285851
1,서귀포시,이에스제주리조트,제주특별자치도 서귀포시 1100로 501,126.456559,33.289778
2,서귀포시,천백 휴양펜션,제주특별자치도 서귀포시 1100로 6,126.431904,33.251449
3,서귀포시,모이라펜션,제주특별자치도 서귀포시 516로 296,126.587219,33.279067
4,서귀포시,빅토르힐 리조트,제주특별자치도 서귀포시 516로277번길 45,126.581437,33.279347


In [9]:
# 좌표 변환 실패 행 삭제 및 인덱스 초기화
df_hotel.dropna(axis=0, inplace=True)
df_hotel.reset_index(drop = True, inplace=True)
df_hotel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1330 entries, 0 to 1329
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   행정시     1330 non-null   object 
 1   업소명     1330 non-null   object 
 2   소재지     1330 non-null   object 
 3   X       1330 non-null   float64
 4   Y       1330 non-null   float64
dtypes: float64(2), object(3)
memory usage: 52.1+ KB


In [10]:
# 파일 저장
df_hotel.to_csv("processedData/hotel.csv", index=False)

## 상비의약품판매업소 데이터

In [11]:
df_store = pd.read_csv("제주특별자치도_안전상비의약품판매업소정보.csv", encoding='cp949')

In [12]:
df_store = df_store[df_store['영업상태명'] == "영업/정상"]
df_store = df_store[['사업장명', '도로명주소']]
df_store.dropna(axis=0, inplace=True)
df_store.drop_duplicates(ignore_index=True, inplace=True)
df_store.reset_index(drop = True, inplace=True)
df_store.info()
df_store.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852 entries, 0 to 851
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   사업장명    852 non-null    object
 1   도로명주소   852 non-null    object
dtypes: object(2)
memory usage: 13.4+ KB


,사업장명,도로명주소
0,씨유제주이도정든점,제주특별자치도 제주시 정든로 40(이도이동)
1,씨유 제주탑동서문점,제주특별자치도 제주시 탑동로 136 (용담일동)
2,씨유 제주종달점,제주특별자치도 제주시 구좌읍 일주동로 3719
3,지에스25연동신라점,"제주특별자치도 제주시 노연로 61, 다울하우스 1층 105호 (연동)"
4,씨유제주용담중앙점,"제주특별자치도 제주시 남성로2길 3, 1층 (용담일동)"


In [13]:
# 좌표 변환
dfAddrToCoordMap(df_store, "도로명주소")

주소: 제주특별자치도 제주시 정든로 40(이도이동) => 경도: 126.542810498726, 위도: 33.497398133535
주소: 제주특별자치도 제주시 탑동로 136 (용담일동) => 경도: 126.518293238834, 위도: 33.5131420546055
주소: 제주특별자치도 제주시 구좌읍 일주동로 3719 => 경도: 126.893930977912, 위도: 33.4928127909798
주소: 제주특별자치도 제주시 노연로 61, 다울하우스 1층 105호 (연동) => 경도: 126.486786853647, 위도: 33.4861786208647
주소: 제주특별자치도 제주시 남성로2길 3, 1층 (용담일동) => 경도: 126.513353131432, 위도: 33.5014214535981
주소: 제주특별자치도 제주시 노연로 30, 1층 (노형동, 에코빌리지) => 경도: 126.483391478798, 위도: 33.4858742008363
주소: 제주특별자치도 제주시 화삼로 107, 상가동 101호 (도련일동, 제주삼화사랑으로부영6차) => 경도: 126.584333422234, 위도: 33.5158012135932
주소: 제주특별자치도 제주시 서광로 283, 1층 (이도일동) => 경도: 126.526819773118, 위도: 33.5005285656872
주소: 제주특별자치도 제주시 도근내길 24, 1층 (외도일동) => 경도: 126.433459305054, 위도: 33.4923184653298
주소: 제주특별자치도 제주시 신광로8길 22, 1층 (연동) => 경도: 126.487830477518, 위도: 33.4871051769288
주소: 제주특별자치도 제주시 노형로 379, 1층 (노형동) => 경도: 126.475999755697, 위도: 33.4840345744526
주소: 제주특별자치도 제주시 정존7길 37 (노형동) => 경도: 126.473049665477, 위도: 33.485961930454
주소: 제주특별자치도 제주시 성화로

In [14]:
df_store.dropna(axis=0, inplace=True)
df_store.reset_index(drop = True, inplace=True)
df_store.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 840 entries, 0 to 839
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   사업장명    840 non-null    object 
 1   도로명주소   840 non-null    object 
 2   X       840 non-null    float64
 3   Y       840 non-null    float64
dtypes: float64(2), object(2)
memory usage: 26.4+ KB


In [15]:
# 파일 저장
df_store.to_csv("processedData/store.csv", index=False)

## 약국 데이터

In [16]:
df_pharmacy = pd.read_csv("전국약국표준데이터.csv", encoding='cp949')

C:\Users\jhpark\AppData\Local\Temp/ipykernel_3156/3508340190.py:1: DtypeWarning: Columns (5,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df_pharmacy = pd.read_csv("전국약국표준데이터.csv", encoding='cp949')


In [17]:
df_pharmacy.info()
df_pharmacy.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65497 entries, 0 to 65496
Data columns (total 31 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   번호           65497 non-null  int64  
 1   개방서비스명       65497 non-null  object 
 2   개방서비스아이디     65497 non-null  object 
 3   개방자치단체코드     65497 non-null  int64  
 4   관리번호         65497 non-null  object 
 5   인허가일자        65497 non-null  object 
 6   인허가취소일자      690 non-null    float64
 7   영업상태구분코드     65497 non-null  int64  
 8   영업상태명        65497 non-null  object 
 9   상세영업상태코드     65497 non-null  int64  
 10  상세영업상태명      65497 non-null  object 
 11  폐업일자         35225 non-null  float64
 12  휴업시작일자       788 non-null    float64
 13  휴업종료일자       314 non-null    float64
 14  재개업일자        0 non-null      float64
 15  소재지전화        56329 non-null  object 
 16  소재지면적        0 non-null      float64
 17  소재지우편번호      39745 non-null  float64
 18  소재지전체주소      61282 non-null  object 
 19  도로명전

,번호,개방서비스명,개방서비스아이디,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태구분코드,영업상태명,상세영업상태코드,...,사업장명,최종수정시점,데이터갱신구분,데이터갱신일자,업태구분명,좌표정보(x),좌표정보(y),약국영업면적,지정일자,Unnamed: 30
0,1,약국,01_01_06_P,3020000,PHMD119733020033084000002,19730503,NaN,1,영업/정상,18,...,김상돈약국,20090529102252,I,2018-08-31 23:59:59.0,NaN,NaN,NaN,33.75,19730503.0,NaN
1,2,약국,01_01_06_P,3620000,PHMD119873620020084000002,19870530,NaN,1,영업/정상,13,...,란약국,20111215145442,I,2018-08-31 23:59:59.0,NaN,193786.830157,185954.519436,21.51,19870530.0,NaN
2,3,약국,01_01_06_P,3620000,PHMD119873620020084000003,19870629,NaN,1,영업/정상,13,...,메디팜건강약국,20150728112253,I,2018-08-31 23:59:59.0,NaN,194075.244057,184984.418601,21.19,19870629.0,NaN
3,4,약국,01_01_06_P,3620000,PHMD120033620020084000001,20030317,NaN,1,영업/정상,13,...,좋은약국,20150727163504,I,2018-08-31 23:59:59.0,NaN,193146.400498,186088.106546,0.00,20030317.0,NaN
4,5,약국,01_01_06_P,3620000,PHMD120033620020084000002,20030325,NaN,1,영업/정상,13,...,라포약국,20220120104712,U,2022-01-22 02:40:00.0,NaN,194270.107173,184985.278274,0.00,20030325.0,NaN


In [18]:
df_pharmacy = df_pharmacy[["사업장명", "영업상태구분코드", "도로명전체주소", "좌표정보(x)", "좌표정보(y)"]]
df_pharmacy.dropna(axis=0, inplace=True)
df_pharmacy = df_pharmacy[df_pharmacy['영업상태구분코드'] == 1 & df_pharmacy['도로명전체주소'].str.contains("제주")]
df_pharmacy.drop_duplicates(ignore_index=True, inplace=True)

In [19]:
df_pharmacy.reset_index(drop = True, inplace=True)
df_pharmacy = df_pharmacy[["사업장명", "도로명전체주소"]]
df_pharmacy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294 entries, 0 to 293
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   사업장명     294 non-null    object
 1   도로명전체주소  294 non-null    object
dtypes: object(2)
memory usage: 4.7+ KB


In [20]:
# 좌표 변환
dfAddrToCoordMap(df_pharmacy, "도로명전체주소")

주소: 제주특별자치도 제주시 도령로 87, 현대오피스텔 1층 106,107호 (연동) => 경도: 126.487275468846, 위도: 33.4908688308155
주소: 제주특별자치도 서귀포시 일주동로 9213, GS25 1층 6호 (법환동) => 경도: 126.508491059708, 위도: 33.248712252659
주소: 제주특별자치도 제주시 일주동로 366, 1층 (삼양이동) => 경도: 126.584080157623, 위도: 33.5208841725461
주소: 제주특별자치도 제주시 1100로 3347, 1층 (노형동) => 경도: 126.479605628771, 위도: 33.4848372184625
주소: 제주특별자치도 제주시 연삼로 305, 101호 (도남동) => 경도: 126.525167130007, 위도: 33.4923853689194
주소: 제주특별자치도 제주시 화삼북로 63, 삼화빌딩 1층 102호 (삼양이동) => 경도: 126.579866759828, 위도: 33.5174991708227
주소: 제주특별자치도 제주시 애월읍 하귀9길 2, 1층 106호 => 경도: 126.414565502113, 위도: 33.4864182898895
주소: 제주특별자치도 제주시 중앙로 319-1, 1층 (이도이동) => 경도: 126.535990432468, 위도: 33.4927078468873
주소: 제주특별자치도 제주시 구좌읍 해맞이해안로 486, 월정애비뉴 101호 => 경도: 126.798870946339, 위도: 33.5547235169193
주소: 제주특별자치도 서귀포시 중앙로 113 (서홍동) => 경도: 126.560488355674, 위도: 33.25480137328
주소: 제주특별자치도 제주시 서광로 297, 1층 (이도일동) => 경도: 126.528208185944, 위도: 33.500735275902
주소: 제주특별자치도 제주시 애월읍 곽지7길 1, 2층 => 경도: 126.308399811765, 위도: 33.450052

In [21]:
df_pharmacy.dropna(axis=0, inplace=True)
df_pharmacy.reset_index(drop = True, inplace=True)
df_pharmacy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291 entries, 0 to 290
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   사업장명     291 non-null    object 
 1   도로명전체주소  291 non-null    object 
 2   X        291 non-null    float64
 3   Y        291 non-null    float64
dtypes: float64(2), object(2)
memory usage: 9.2+ KB


In [22]:
# 파일 저장
df_pharmacy.to_csv("processedData/pharmacy.csv", index=False)

## 응급실 데이터

In [23]:
df_emergency = pd.read_csv("제주특별자치도_응급의료기관현황_20211231.csv", encoding='cp949')

In [24]:
df_emergency = df_emergency[["의료기관", "소재지"]]
df_emergency

,의료기관,소재지
0,제주한라병원,제주특별자치도 제주시 도령로 65
1,제주대학교병원,제주특별자치도 제주시 아란13길 15
2,한마음병원,제주특별자치도 제주시 연신로 52
3,서귀포의료원,제주특별자치도 서귀포시 장수로 47
4,중앙병원,제주특별자치도 제주시 월랑로 91
5,한국병원,제주특별자치도 제주시 서광로 193
6,연동365일의원,제주특별자치도 제주시 연북로 99
7,탑동365일의원,제주특별자치도 제주시 탑동로 24


In [25]:
dfAddrToCoordMap(df_emergency, "소재지")

주소: 제주특별자치도 제주시 도령로 65 => 경도: 126.484919722369, 위도: 33.4902661749243
주소: 제주특별자치도 제주시 아란13길 15 => 경도: 126.545201028535, 위도: 33.4672085209581
주소: 제주특별자치도 제주시 연신로 52 => 경도: 126.546238571767, 위도: 33.4962868079834
주소: 제주특별자치도 서귀포시 장수로 47 => 경도: 126.562948872895, 위도: 33.2559809912099
주소: 제주특별자치도 제주시 월랑로 91 => 경도: 126.470480704295, 위도: 33.4923211704292
주소: 제주특별자치도 제주시 서광로 193 => 경도: 126.516729196278, 위도: 33.5003915262127
주소: 제주특별자치도 제주시 연북로 99  => 경도: 126.492005938839, 위도: 33.4821139255537
주소: 제주특별자치도 제주시 탑동로 24  => 경도: 126.522460881414, 위도: 33.517560202795
총 데이터 개수: 8, 성공: 8, 실패: 0


In [26]:
df_emergency

,의료기관,소재지,X,Y
0,제주한라병원,제주특별자치도 제주시 도령로 65,126.484920,33.490266
1,제주대학교병원,제주특별자치도 제주시 아란13길 15,126.545201,33.467209
2,한마음병원,제주특별자치도 제주시 연신로 52,126.546239,33.496287
3,서귀포의료원,제주특별자치도 서귀포시 장수로 47,126.562949,33.255981
4,중앙병원,제주특별자치도 제주시 월랑로 91,126.470481,33.492321
5,한국병원,제주특별자치도 제주시 서광로 193,126.516729,33.500392
6,연동365일의원,제주특별자치도 제주시 연북로 99,126.492006,33.482114
7,탑동365일의원,제주특별자치도 제주시 탑동로 24,126.522461,33.517560


In [27]:
# 파일 저장
df_emergency.to_csv("processedData/emergency.csv", index=False)

## 학교 데이터

In [28]:
df_school = pd.read_csv("제주특별자치도_학교현황_20200930.csv", encoding='cp949')

In [29]:
df_school = df_school[["학교", "자치구", "남녀공학구분", "주소"]]
df_school.drop_duplicates(ignore_index=True, inplace=True)
df_school.info()
df_school.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321 entries, 0 to 320
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   학교      321 non-null    object
 1   자치구     321 non-null    object
 2   남녀공학구분  321 non-null    object
 3   주소      321 non-null    object
dtypes: object(4)
memory usage: 10.2+ KB


,학교,자치구,남녀공학구분,주소
0,제주대학교사범대학부설고등학교,제주시,남여공학,제주특별자치도 제주시 용담로 65
1,대정고등학교,서귀포시,남,제주특별자치도 서귀포시 대정읍 일주서로2498번길 21
2,대정여자고등학교,서귀포시,여,제주특별자치도 서귀포시 대정읍 대한로88번길 82-22
3,서귀포고등학교,서귀포시,남,제주특별자치도 서귀포시 중앙로 140
4,서귀포산업과학고등학교,서귀포시,남여공학,제주특별자치도 서귀포시 516로 501


In [30]:
dfAddrToCoordMap(df_school, "주소")

주소: 제주특별자치도 제주시 용담로 65 => 경도: 126.5102401196, 위도: 33.5137606145563
주소: 제주특별자치도 서귀포시 대정읍 일주서로2498번길 21 => 경도: 126.259500374588, 위도: 33.2319799989913
주소: 제주특별자치도 서귀포시 대정읍 대한로88번길 82-22 => 경도: 126.253314209595, 위도: 33.2291887736802
주소: 제주특별자치도 서귀포시 중앙로 140 => 경도: 126.563108014727, 위도: 33.2570172520025
주소: 제주특별자치도 서귀포시 516로 501 => 경도: 126.594767157466, 위도: 33.2976458557396
주소: 제주특별자치도 서귀포시 태평로 80 => 경도: 126.530797586105, 위도: 33.2434957468657
주소: 제주특별자치도 서귀포시 성산읍 한도로 79-17 => 경도: 126.91516666622, 위도: 33.4702032373211
주소: 제주특별자치도 제주시 구좌읍 세종로 26 => 경도: 126.862739001471, 위도: 33.516197330754
주소: 제주특별자치도 제주시 애월읍 일주서로 6372-20 => 경도: 126.333396217069, 위도: 33.4602188962729
주소: 제주특별자치도 제주시 1100로 3213 => 경도: 126.481670922303, 위도: 33.4723734166954
주소: 제주특별자치도 제주시 산록북로 421-1 => 경도: 126.531444515992, 위도: 33.4266199894809
주소: 제주특별자치도 제주시 동문로 89-1 => 경도: 126.536352998468, 위도: 33.514660180355
주소: 제주특별자치도 제주시 애월읍 고성남길 34 => 경도: 126.425979138844, 위도: 33.4339757479804
주소: 제주특별자치도 제주시 진군길 22 => 경도: 126.4807053

In [31]:
df_school.dropna(axis=0, inplace=True)
df_school.reset_index(drop = True, inplace=True)
df_school.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   학교      320 non-null    object 
 1   자치구     320 non-null    object 
 2   남녀공학구분  320 non-null    object 
 3   주소      320 non-null    object 
 4   X       320 non-null    float64
 5   Y       320 non-null    float64
dtypes: float64(2), object(4)
memory usage: 15.1+ KB


In [32]:
# 파일 저장
df_school.to_csv("processedData/school.csv", index=False)

#### 인구 데이터 (미완)

In [33]:
# 데이터 로드
df_population = pd.read_csv("제주특별자치도_읍면동별세대및인구_2021.csv", encoding='cp949')
df_elderlyPopulation = pd.read_csv("제주특별자치도_읍면동별노인인구현황_20210630.csv", encoding='cp949')

In [34]:
df_population.head()

,행정시별(1),행정시별(2),2021,2021.1,2021.2,2021.3,2021.4,2021.5,2021.6,2021.7,2021.8,2021.9
0,행정시별(1),행정시별(2),제주 인구(계) (명),제주 인구(남) (명),제주 인구(여) (명),한국인 세대수 (세대),한국인(계) (명),한국인(남) (명),한국인(여) (명),외국인(계) (명),외국인(남) (명),외국인(여) (명)
1,제주특별자치도,소계,697476,350822,346654,307529,676759,339177,337582,20717,11645,9072
2,제주시,소계,506843,253974,252869,219978,493096,246494,246602,13747,7480,6267
3,제주시,한 림 읍,24291,12917,11374,10641,21248,10908,10340,3043,2009,1034
4,제주시,애 월 읍,38483,19684,18799,17276,37532,19209,18323,951,475,476


In [35]:
df_elderlyPopulation.head()

,해당연도,행정시,읍면동,65세 이상 인구,65세 이상 비율,65세 이상 순위,80세 이상 인구,80세 이상 비율,80세 이상 순위,85세 이상 인구,85세 이상 비율,85세 이상 순위,100세 이상 인구,100세 이상 비율,100세 이상 순위,데이터기준일자
0,2020,제주시,한림읍,4611,21.7,6,1396,6.6,6,673,3.2,6,28,0.13,3,2021-06-30
1,2020,제주시,애월읍,6398,17.3,14,1937,5.2,10,929,2.5,12,14,0.04,15,2021-06-30
2,2020,제주시,구좌읍,4004,25.8,4,1392,9.0,4,683,4.4,4,8,0.05,10,2021-06-30
3,2020,제주시,조천읍,4732,18.7,11,1376,5.4,8,652,2.6,10,12,0.05,11,2021-06-30
4,2020,제주시,한경면,2488,27.6,3,892,9.9,2,452,5.0,1,9,0.10,5,2021-06-30
